In [ ]:
!pip install pip
!pip install tensorflow
!pip install numpy
!pip install matplotlib

Data Preprocessing
Reads all STL files in folder called 'STLFiles' and convertes to voxelized png images before turning all images into one long png file for each STL file.

In [ ]:
from PIL import Image
import os
import shutil
import stltovoxel

# Define the folder containing the STL files
stl_folder = 'STLFiles'
output_folder = 'VoxelGraphs'
resolution = 150
target_width = 100
target_height = 100

# Iterate through all files in the stl_folder directory
for stlfileInput in os.listdir(stl_folder):
    if stlfileInput.lower().endswith(".stl"):
        # Construct the full path to the STL file
        stlfileInput = os.path.join(stl_folder, stlfileInput)

        # Convert STL file to PNG using stltovoxel
        stltovoxel.convert_file(stlfileInput, 'output.png', resolution=resolution)

        # Get a list of all files in the current directory
        files = os.listdir()

        # Resize and center images in the current directory
        resized_images = []

        for file in files:
            if file.lower().endswith(".png"):
                image_path = os.path.join(os.getcwd(), file)
                img = Image.open(image_path)

                # Resize without distortion and center the image
                original_width, original_height = img.size

                if original_width / target_width > original_height / target_height:
                    new_width = target_width
                    new_height = int(original_height * (target_width / original_width))
                else:
                    new_width = int(original_width * (target_height / original_height))
                    new_height = target_height

                resized_image = Image.new('RGB', (target_width, target_height), (0, 0, 0))
                resized_image.paste(img.resize((new_width, new_height)), ((target_width - new_width) // 2, (target_height - new_height) // 2))

                resized_images.append(resized_image)

        # Combine all resized and centered images into a single image
        combined_image = Image.new('RGB', (target_width, target_height * len(resized_images)))

        for i, img in enumerate(resized_images):
            combined_image.paste(img, (0, i * target_height))

        # Save the combined image with the same name as the STL file in the VoxelGraphs directory
        combined_image_name = os.path.splitext(os.path.basename(stlfileInput))[0] + '_combined.png'
        combined_image_path = os.path.join(output_folder, combined_image_name)
        combined_image.save(combined_image_path)

        # Delete other PNG files in the current directory
        for file in files:
            if file.lower().endswith(".png") and file != combined_image_name:
                file_path = os.path.join(os.getcwd(), file)
                os.remove(file_path)

print("All images processed.")


Building Network Model
Splits data into training and test data
Creates neural network and trains on data from storage

In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras import layers, models

# Define paths
data_directory = 'VoxelGraphs'
batch_size = 1  # You can change this to your desired batch size

#resolution = 150
image_width = target_width
image_height = target_height

learning_rate = 1

#------------------------------------------------------

image_height = image_height * (resolution + 2)

# Create an ImageDataGenerator for data augmentation and normalization
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # You can adjust the validation split
)

# Create a training dataset
train_dataset = datagen.flow_from_directory(
    data_directory,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
    seed=123  # Set a seed for reproducibility
)

# Create a validation dataset
validation_dataset = datagen.flow_from_directory(
    data_directory,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
    seed=123  # Set a seed for reproducibility
)

# Build a simple convolutional neural network (CNN) model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_width, image_height, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification output
])

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_dataset,
    steps_per_epoch=train_dataset.samples // batch_size,
    validation_data=validation_dataset,
    validation_steps=validation_dataset.samples // batch_size,
    epochs=5
)


------------------------------